In [ ]:
# Reference - For BAHDANAU METHOD and others as well - https://towardsdatascience.com/attention-seq2seq-with-pytorch-learning-to-invert-a-sequence-34faf4133e53
# Reference - For DOT METHOD - https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [ ]:
import pandas as pd
import numpy as np
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/DLassignment3

hi.translit.sampled.train.tsv


In [ ]:
data = pd.read_csv ("/content/drive/MyDrive/DLassignment3/hi.translit.sampled.train.tsv", sep = '\t')

In [ ]:
data.rename(columns = {'अं' : 'Hindi', 'an' : 'English'}, inplace = True)
data

,Hindi,English,3
0,अंकगणित,ankganit,3
1,अंकल,uncle,4
2,अंकुर,ankur,4
3,अंकुरण,ankuran,3
4,अंकुरित,ankurit,3
...,...,...,...
44198,ह्वेनसांग,hiuentsang,1
44199,ह्वेनसांग,hsuantsang,1
44200,ह्वेनसांग,hyensang,1
44201,ह्वेनसांग,xuanzang,1


In [ ]:
  len(data)

44203

In [ ]:
input_texts=data['English']
target_texts=data['Hindi']

In [ ]:
input_texts=list(input_texts)
target_texts=list(target_texts)

In [ ]:
alphabets = set()
characters = set()
alphabets.add(' ')
alphabets.add('<sos>')
alphabets.add('<eos>')
characters.add(' ')
characters.add('<sos>')
characters.add('<eos>')

In [ ]:
i=0
for input_text in input_texts:
  #print(input_text)
  try:
    for char in input_text:
      if char not in alphabets:
        alphabets.add(char)
  except:
    print("********************************************************************************************************************************")
    input_texts.remove(input_text)
    target_texts.remove(target_texts[i])
  i=i+1;

********************************************************************************************************************************
********************************************************************************************************************************


In [ ]:
for input_text in target_texts:
  #print(input_text)
  try:
    for char in input_text:
      if char not in characters:
        characters.add(char)
  except:print(input_text)

In [ ]:
# alphabets=['\t','\n']
# # for i in range(65,91):
# #   alphabets.append(chr(i))
# for i in range(97,123):
#   alphabets.append(chr(i))
# alphabets.add(' ')
alphabets = sorted(list(alphabets))

In [ ]:
cnt = 0
for val in alphabets:
  print(cnt," ",val)
  cnt+=1

0    
1   <eos>
2   <sos>
3   a
4   b
5   c
6   d
7   e
8   f
9   g
10   h
11   i
12   j
13   k
14   l
15   m
16   n
17   o
18   p
19   q
20   r
21   s
22   t
23   u
24   v
25   w
26   x
27   y
28   z


In [ ]:
# characters=['\t','\n']
# char_list=[codepoint for codepoint in range(0x0900, 0x0980)]
# for i in char_list:
#   characters.append(chr(i))
# characters.add(' ')
characters = sorted(list(characters))

In [ ]:
cnt = 0
for val in characters:
  print(cnt," ",val)
  cnt+=1

0    
1   <eos>
2   <sos>
3   ँ
4   ं
5   ः
6   अ
7   आ
8   इ
9   ई
10   उ
11   ऊ
12   ऋ
13   ए
14   ऐ
15   ऑ
16   ओ
17   औ
18   क
19   ख
20   ग
21   घ
22   ङ
23   च
24   छ
25   ज
26   झ
27   ञ
28   ट
29   ठ
30   ड
31   ढ
32   ण
33   त
34   थ
35   द
36   ध
37   न
38   प
39   फ
40   ब
41   भ
42   म
43   य
44   र
45   ल
46   व
47   श
48   ष
49   स
50   ह
51   ़
52   ा
53   ि
54   ी
55   ु
56   ू
57   ृ
58   ॅ
59   े
60   ै
61   ॉ
62   ो
63   ौ
64   ्
65   ॐ


In [ ]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print(max_encoder_seq_length,",",max_decoder_seq_length)

20 , 19


In [ ]:
sample_out=[]
sample_in=[]
for i in range(len(data)):
  word_o=data.iloc[i][0]
  word_i=data.iloc[i][1]
  #print("word",word)
  word_vec_o=np.zeros(20, dtype=int)
  word_vec_i=np.zeros(20, dtype=int)
  
  try:
    lo=list(word_o)
    x=len(lo)
    li=list(word_i)
    y=len(li)
    #print("l",l)
    word_vec_o[0] = 2
    cnt = 1
    for j in range(0,x):
      cnt += 1
      #print("l[j]",l[j])
      ind=characters.index(lo[j])
      word_vec_o[j+1] = ind
    word_vec_o[cnt] = 1
    
    word_vec_i[0] = 2
    cnt = 1
    for j in range(0,y):
      cnt += 1
      #print("l[j]",l[j])
      ind=alphabets.index(li[j])
      word_vec_i[j+1] = ind
    word_vec_i[cnt] = 1
    sample_out.append(word_vec_o)
    sample_in.append(word_vec_i)
  except:
    print("error");
  #print(i)

error
error
error
error
error
error
error


In [ ]:
data.iloc[0][1]

'ankganit'

In [ ]:
sample_in[0]

array([ 2,  3, 16, 13,  9,  3, 16, 11, 22,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [ ]:
data.iloc[0][0]

'अंकगणित'

In [ ]:
sample_out[0]

array([ 2,  6,  4, 18, 20, 32, 53, 33,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [ ]:
tmp_x_train_index = random.sample([i for i in range(np.array(sample_in).shape[0])],math.ceil(np.array(sample_in).shape[0]*0.7))
tmp_x_train = []
tmp_x_val = []
tmp_y_train = []
tmp_y_val = []
for i in tmp_x_train_index:
  tmp_x_train.append(sample_in[i])
  tmp_y_train.append(sample_out[i])
for i in range(np.array(sample_in).shape[0]):
  if i not in tmp_x_train_index:
    tmp_x_val.append(sample_in[i])
    tmp_y_val.append(sample_out[i])

# x_train = np.array(sample_in[:math.ceil(np.array(sample_in).shape[0]*0.7)])
# y_train = np.array(sample_out[:math.ceil(np.array(sample_out).shape[0]*0.7)])
# x_val = np.array(sample_in[math.ceil(np.array(sample_in).shape[0]*0.7):])
# y_val = np.array(sample_out[math.ceil(np.array(sample_out).shape[0]*0.7):])
# x_train = np.array(sample_in)
# y_train = np.array(sample_out)

In [ ]:
for val in tmp_x_train[0]:
  print(alphabets[int(val)],end="")
print()
for val in tmp_y_train[0]:
  print(characters[int(val)],end="")
print()
for val in tmp_x_val[0]:
  print(alphabets[int(val)],end="")
print()
for val in tmp_y_val[0]:
  print(characters[int(val)],end="")
print()

<sos>agantuk<eos>           
<sos>आगंतुक<eos>            
<sos>ankganit<eos>          
<sos>अंकगणित<eos>           


In [ ]:
# x_train = np.array(tmp_x_train)
# y_train = np.array(tmp_y_train)
# x_val = np.array(tmp_x_val)
# y_val = np.array(tmp_y_val)
x_train = tmp_x_train
y_train = tmp_y_train
x_val = tmp_x_val
y_val = tmp_y_val

In [ ]:
torch.tensor(x_train[32*0:32*1][:]).T.shape

torch.Size([20, 32])

In [ ]:
emb_size = 300
hid_size = 128
n_layer = 1

**ENCODER MODEL**

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(self.hidden_size, self.hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x, hidden_state, cell_state):
    # print("EncoderModel:input :",x.unsqueeze(0).shape)
    x = x.unsqueeze(0)

    # Shape -----------> (20, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    outputs = embedding
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(outputs, (hidden_state, cell_state))

    return outputs, hidden_state, cell_state

input_size_encoder = len(alphabets)
encoder_embedding_size = emb_size
hidden_size = hid_size
num_layers = n_layer
encoder_dropout = float(0.5)

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(29, 128)
  (LSTM): LSTM(128, 128, dropout=0.5)
)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


**ATTENTION DECODER LSTM**

**DOT METHOD**

In [ ]:
# DOT METHOD
class AttentionDecoderLSTM_dot(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, max_length, num_layers, p, output_size):
    super(AttentionDecoderLSTM_dot, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Max length??? --> Can be considered as max timesteps used?
    self.max_length = max_length

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Softmax Function
    self.softmax = nn.Softmax(dim=1)

    # ReLU Function
    self.relu = nn.ReLU()

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)

    # attn -> Previous Hidden + Current Embedding
    # self.attn = nn.Linear(self.hidden_size*2, self.max_length)
    self.attn = nn.Linear(self.hidden_size*3, self.max_length)

    # attn_combine -> bmm(Encoder outputs, softmax(attn))
    self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(self.hidden_size, self.hidden_size, self.num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(self.hidden_size, self.output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state, encoder_outputs):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)
    print("DecoderModel:Input :",x.shape)
    print("DecoderModel:Hidden state :",hidden_state.shape)
    print("DecoderModel:Cell state :",cell_state.shape)
    print("DecoderModel:Encoder Output :",encoder_outputs.shape)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))
    print("DecoderModel:embedding :",embedding.shape)

    concat_emb_state = torch.cat( (embedding[0], hidden_state[0], cell_state[0]),1) # dim = 1 --> inner most vectors are concatinated i.e. (1,32,256)X(1,32,256)X(1,32,256) = (32X768)
    print("DecoderModel:concat embedding + state :",concat_emb_state.shape)

    attn_e_s = self.attn(concat_emb_state) # (32 X 768) X (768 X 20)
    print("DecoderModel:attn embedding + state :",attn_e_s.shape)

    attn_weights = self.softmax(attn_e_s) # (32 x 20)
    print("DecoderModel:attn_weights :",attn_weights.shape) 

    attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs) # (32x1x20) x (32x20x256) = (32x1x256)
    print("DecoderModel:attn_applied :",attn_applied.shape)

    # print(attn_applied[0][0].shape)
    attn_applied_f = torch.zeros(attn_applied.shape[0], attn_applied.shape[2]) # Need to convert (32x1x256) to (32x256)
    print(attn_applied_f.shape)

    for i in range(attn_applied.shape[0]):
      attn_applied_f[i] = attn_applied[i][0]

    emb_attn_cat = torch.cat((embedding[0], attn_applied_f.unsqueeze(0)[0]), 1) # concat (1x32x256) and (1x32x256) with dim=1 --> (32x512)
    print("DecoderModel:(Embedding + attn_applied) :",emb_attn_cat.shape)

    attn_combine = self.attn_combine(emb_attn_cat).unsqueeze(0) # (1x32x512) --> (1x32x256)
    print("DecoderModel:attn_combine :",attn_combine.shape)

    outputs = self.relu(attn_combine)

    # Shape --> outputs (1, 32, 256) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 256) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(outputs, (hidden_state, cell_state))


    # Shape --> predictions (1, 32, 66) [ 1, batch_size , output_size]
    predictions = self.softmax(self.fc(outputs))
    print("DecoderModel:Predictions : ",predictions.shape)

    # Shape --> predictions (32, 66) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(characters)
decoder_embedding_size = emb_size
hidden_size = hid_size
num_layers = n_layer
decoder_dropout = float(0.5)
output_size = len(characters)
max_length = 20

decoder_lstm_dot = AttentionDecoderLSTM_dot(input_size_decoder, decoder_embedding_size,
                           hidden_size, max_length, num_layers, decoder_dropout, output_size)
print(decoder_lstm_dot)

AttentionDecoderLSTM_dot(
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
  (relu): ReLU()
  (embedding): Embedding(66, 128)
  (attn): Linear(in_features=384, out_features=20, bias=True)
  (attn_combine): Linear(in_features=256, out_features=128, bias=True)
  (LSTM): LSTM(128, 128, dropout=0.5)
  (fc): Linear(in_features=128, out_features=66, bias=True)
)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


**BAHDANAU METHOD**

In [ ]:
# BAHDANAU METHOD
class AttentionDecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, max_length, num_layers, p, output_size):
    super(AttentionDecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # self.batch_size = batch_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Max length??? --> Can be considered as max timesteps used?
    self.max_length = max_length

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Softmax Function
    self.softmax = nn.Softmax(dim=1)

    # Tanh Function
    self.tanh = nn.Tanh()

    # Weighted encoder_output and hidden layer (W*hid_l + U*enc_out)
    self.Wa = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
    self.Ua = nn.Linear(self.hidden_size, self.hidden_size, bias=False)

    # while doing bmm with weighted out
    # self.va = nn.Parameter(torch.FloatTensor(self.batch_size, self.hidden_size))

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM_1 = nn.LSTM(self.hidden_size, self.hidden_size, self.num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(self.hidden_size*2, self.output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state, encoder_outputs,batch_size):
    # self.batch_size = batch_size
    # self.va = nn.Parameter(torch.FloatTensor(self.batch_size, self.hidden_size))

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)
    # print("DecoderModel:Input :",x.shape)
    # print("DecoderModel:Hidden state :",hidden_state.shape)
    # print("DecoderModel:Cell state :",cell_state.shape)
    # print("DecoderModel:Encoder Output :",encoder_outputs.shape)

    # Shape -----------> (1, 32, 256) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))
    # print("DecoderModel:embedding :",embedding.shape)


    # Shape ----> output, hidden, cell ---> (1,32,256)
    L, (hidden_state, cell_state) = self.LSTM_1(embedding, (hidden_state, cell_state))
    # print("DecoderModel:lstm_1:outputs :",L.shape)
    # print("DecoderModel:lstm_1:hidden :",hidden_state.shape)
    # print("DecoderModel:lstm_1:cell :",cell_state.shape)

    L = L.squeeze(0).unsqueeze(1) # Shape --> (32,1,256)
    # print("DecoderModel:lstm_1:outputs :",L.shape)

    # tanh(W*hidden_layer + U*encoder_output) --> (32,20,256)
    # hidden_layer --> (32,1,256)
    # encoder_output --> (32,20,256)
    out = self.tanh(self.Wa(L) + self.Ua(encoder_outputs))
    # print("DecoderModel:weighted out :",out.shape)

    # (32x20x256) x (32x256x1) ---> (32x20x1) --> (32x20)
    wts = torch.bmm(out, nn.Parameter(torch.FloatTensor(batch_size, self.hidden_size)).unsqueeze(2)).squeeze(-1)
    wts = F.softmax(wts, dim=-1)
    # print("DecoderModel:weights :",wts.shape)

    # Weighted sum of encoder outputs (32x1x20) x (32x20x256) --> (32x1x256) --> (32x256)
    context = torch.bmm(wts.unsqueeze(1), encoder_outputs).squeeze(1)
    # print("DecoderModel:context :",context.shape)

    # Linearize context and attn_comb(Embedding, Previous hidden layer)
    # step 1 : (32x256) + (32x256) --> (32x512)
    # step 2 : (32x512) --> (32x256)
    predictions = self.softmax(self.fc(torch.cat((L.squeeze(1), context),1)))
    # print("DecoderModel:Predictions : ",predictions.shape)

    return predictions, hidden_state, cell_state

input_size_decoder = len(characters)
decoder_embedding_size = emb_size
hidden_size = hid_size
num_layers = n_layer
decoder_dropout = float(0.5)
output_size = len(characters)
max_length = 20

decoder_lstm = AttentionDecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, max_length, num_layers, decoder_dropout, output_size)
print(decoder_lstm)

AttentionDecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
  (tanh): Tanh()
  (Wa): Linear(in_features=128, out_features=128, bias=False)
  (Ua): Linear(in_features=128, out_features=128, bias=False)
  (embedding): Embedding(66, 128)
  (LSTM_1): LSTM(128, 128, dropout=0.5)
  (fc): Linear(in_features=256, out_features=66, bias=True)
)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


**ATTENTION Seq2Seq**

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target,num_layers, hidden_size,max_length = 20, tfr=0.5):
    # Shape - Source : (20, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (20, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(characters)
    
    # Shape --> outputs (20, 32, 66) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size)
    # print("Model:Output : ",outputs.shape)

    temp_encoder_outputs = torch.zeros(max_length, batch_size, hidden_size)
    # print("Model:Temp Encoder_outputs",temp_encoder_outputs.shape)

    encoder_outputs = torch.zeros(batch_size,max_length, hidden_size)
    # print("Model:Encoder_outputs",encoder_outputs.shape)

    hidden_state_encoder = torch.zeros(num_layers, batch_size, hidden_size) # 1 X 32 X 256
    cell_state_encoder = torch.zeros(num_layers, batch_size, hidden_size) # 1 X 32 X 256
    # print("Model:InitStateLayer : ",hidden_state_encoder.shape,cell_state_encoder.shape)

    # Shape --> (hs, cs) (2, 32, 256) ,(2, 32, 256) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    for ei in range(target_len):
        encoder_output, hidden_state_encoder, cell_state_encoder = self.Encoder_LSTM(source[ei], hidden_state_encoder, cell_state_encoder)
        temp_encoder_outputs[ei] = encoder_output[0]
        # print(ei," Model:Encoder Outputs and states :")
        # print(encoder_output.shape)
        # print(hidden_state_encoder.shape)
        # print(cell_state_encoder.shape)
    
    for i in range(batch_size):
      for j in range(max_length):
        encoder_outputs[i][j] = temp_encoder_outputs[j][i]
    
    # hidden_state_encoder, cell_state_encoder = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>
    # print("Model:Target : ", x.shape)

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state_decoder, cell_state_decoder = self.Decoder_LSTM(x, hidden_state_encoder, cell_state_encoder, encoder_outputs,batch_size)
      # print(ei," Model:Decoder Outputs and states :")
      # print(output.shape)
      # print(hidden_state_decoder.shape)
      # print(cell_state_decoder.shape)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs

model_lstm = Seq2Seq(encoder_lstm,decoder_lstm)
print(model_lstm)

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(29, 128)
    (LSTM): LSTM(128, 128, dropout=0.5)
  )
  (Decoder_LSTM): AttentionDecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (softmax): Softmax(dim=1)
    (tanh): Tanh()
    (Wa): Linear(in_features=128, out_features=128, bias=False)
    (Ua): Linear(in_features=128, out_features=128, bias=False)
    (embedding): Embedding(66, 128)
    (LSTM_1): LSTM(128, 128, dropout=0.5)
    (fc): Linear(in_features=256, out_features=66, bias=True)
  )
)


In [ ]:
torch.zeros(32,20).T.unsqueeze(1).shape

torch.Size([20, 1, 32])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_lstm.parameters(), lr=0.001, momentum=0.9)

In [ ]:
print(torch.tensor(x_train).shape[0])
print(math.floor(torch.tensor(x_train).shape[0]/32))
print(math.floor(torch.tensor(x_train).shape[0]/32)*32)

num_of_batches = math.floor(torch.tensor(x_train).shape[0]/32)

30938
966
30912


In [ ]:
def translate_word(model, word_e,word_h,num_layers,max_length=20):    
    # for val in word_e:
    #   print(val, end=", ")
    # print()
    # for val in word_h:
    #   print(val, end=", ")
    print()
    print("Input word : ")
    for val in word_e:
      print(alphabets[val], end="")
    print()
    print("Expected word : ")
    for val in word_h:
      print(characters[val], end="")
    print()
    # print(torch.tensor(word_e).unsqueeze(1).shape)
    # print(torch.tensor(word_h).unsqueeze(1).shape)

    temp_encoder_outputs = torch.zeros(max_length, 1, hidden_size)
    # print("Model:Temp Encoder_outputs",temp_encoder_outputs.shape)

    encoder_outputs = torch.zeros(1,max_length, hidden_size)
    # print("Model:Encoder_outputs",encoder_outputs.shape)

    hidden = torch.zeros(num_layers, 1, hidden_size) # 1 X 32 X 256
    cell = torch.zeros(num_layers, 1, hidden_size) # 1 X 32 X 256
    # Build encoder hidden, cell state
    # print(1)
    # print(torch.tensor([word_e[0]]).shape)
    with torch.no_grad():
      for ei in range(max_length):
        encoder_output, hidden, cell = model.Encoder_LSTM(torch.tensor([word_e[ei]]), hidden, cell)
        temp_encoder_outputs[ei] = encoder_output[0]
    # print(2)
    for j in range(max_length):
      encoder_outputs[0][j] = temp_encoder_outputs[j][0]
    # print(3)
    outputs = [2]
    for i in range(max_length):
        previous_word = torch.tensor([outputs[-1]])

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell, encoder_outputs,1)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)
        # print(output.argmax(1).item())

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == 1:
            break
    # print(4)
    result = ''
    for val in outputs:
      result += characters[val]
    return result

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [ ]:
def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [ ]:
# MAKE CHANGES
epoch_loss = 0.0
num_epochs = 10
best_loss = 999999
best_epoch = -1
sentence1 = "ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster"
ts1 = []

# torch.tensor(x_train[32*0:32*1][:]).T.shape ----> torch.Size([20, 32])

# num_of_batches = 100
step = 0

for epoch in range(num_epochs):
  model_lstm.eval()
  translated_word1 = translate_word(model_lstm, x_train[0], y_train[0],num_layers)
  print(f"Translated word : \n {translated_word1}")
  ts1.append(translated_word1)
  print("Epoch - {} / {}".format(epoch+1, num_epochs))

  model_lstm.train(True)
  # Number of set of batch sizes
  # for batch_idx, batch in enumerate(train_iterator):
  for batch in range(num_of_batches):
    print("Batch : ",batch, end='\r')
    input = torch.tensor(x_train[32*batch:32*(batch+1)][:]).T # of size 20 X 32 (32 -> batch size, 20 -> 1 word max letters)
    target = torch.tensor(y_train[32*batch:32*(batch+1)][:]).T # of size 20 X 32
    # print(input.T.shape)
    # print(target.T.shape)

    # input = batch.src.to(device) 
    # target = batch.trg.to(device) 

    # Pass the input and target for model's forward method
    output = model_lstm(input, target,num_layers,128)
    output = output[1:].reshape(-1, output.shape[2])
    # output = output[:output.shape[0]-1].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    # step += 1
    # epoch_loss += loss.item()
    # writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    save_checkpoint(dict({'model':model_lstm, 'best_loss':best_loss, 'epoch':epoch, 'optimizer':optimizer, 'epoch_loss':epoch_loss}))
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss)

# score = bleu(test_data[1:100], model, german, english, device)
# print(f"Bleu score {score*100:.2f}")


Input word : 
<sos>agantuk<eos>           
Expected word : 
<sos>आगंतुक<eos>            
Translated word : 
 <sos>                    
Epoch - 1 / 10
=> Saving checkpoint
Epoch_Loss - 3.5987019538879395


Input word : 
<sos>agantuk<eos>           
Expected word : 
<sos>आगंतुक<eos>            
Translated word : 
 <sos>                    
Epoch - 2 / 10
Epoch_Loss - 3.598663330078125


Input word : 
<sos>agantuk<eos>           
Expected word : 
<sos>आगंतुक<eos>            
Translated word : 
 <sos>                    
Epoch - 3 / 10
Epoch_Loss - 3.5986709594726562


Input word : 
<sos>agantuk<eos>           
Expected word : 
<sos>आगंतुक<eos>            
Translated word : 
 <sos>                    
Epoch - 4 / 10
Epoch_Loss - 3.598649740219116


Input word : 
<sos>agantuk<eos>           
Expected word : 
<sos>आगंतुक<eos>            
Translated word : 
 <sos>                    
Epoch - 5 / 10
Epoch_Loss - 3.598666191101074


Input word : 
<sos>agantuk<eos>           
Expected word : 
<

**GRU**

In [ ]:
class EncoderGRU(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderGRU, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.GRU = nn.GRU(self.hidden_size, self.hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x, hidden_state):
    # print("EncoderModel:input :",x.unsqueeze(0).shape)
    x = x.unsqueeze(0)

    # Shape -----------> (20, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    outputs = embedding
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, hidden_state = self.GRU(outputs, hidden_state)

    return outputs, hidden_state

input_size_encoder = len(alphabets)
encoder_embedding_size = 300
hidden_size = 128
num_layers = 2
encoder_dropout = float(0.5)

encoder_gru = EncoderGRU(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout)
print(encoder_gru)

In [ ]:
# BAHDANAU METHOD
class AttentionDecoderGRU(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, max_length, num_layers, p, output_size):
    super(AttentionDecoderGRU, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # self.batch_size = batch_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Max length??? --> Can be considered as max timesteps used?
    self.max_length = max_length

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Softmax Function
    self.softmax = nn.Softmax(dim=1)

    # Tanh Function
    self.tanh = nn.Tanh()

    # Weighted encoder_output and hidden layer (W*hid_l + U*enc_out)
    self.Wa = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
    self.Ua = nn.Linear(self.hidden_size, self.hidden_size, bias=False)

    # while doing bmm with weighted out
    # self.va = nn.Parameter(torch.FloatTensor(self.batch_size, self.hidden_size))

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.GRU = nn.GRU(self.hidden_size, self.hidden_size, self.num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(self.hidden_size*2, self.output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, encoder_outputs,batch_size):
    # self.batch_size = batch_size
    # self.va = nn.Parameter(torch.FloatTensor(self.batch_size, self.hidden_size))

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)
    # print("DecoderModel:Input :",x.shape)
    # print("DecoderModel:Hidden state :",hidden_state.shape)
    # print("DecoderModel:Cell state :",cell_state.shape)
    # print("DecoderModel:Encoder Output :",encoder_outputs.shape)

    # Shape -----------> (1, 32, 256) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))
    # print("DecoderModel:embedding :",embedding.shape)


    # Shape ----> output, hidden, cell ---> (1,32,256)
    L, hidden_state = self.GRU(embedding, hidden_state)
    # print("DecoderModel:lstm_1:outputs :",L.shape)
    # print("DecoderModel:lstm_1:hidden :",hidden_state.shape)
    # print("DecoderModel:lstm_1:cell :",cell_state.shape)

    L = L.squeeze(0).unsqueeze(1) # Shape --> (32,1,256)
    # print("DecoderModel:lstm_1:outputs :",L.shape)

    # tanh(W*hidden_layer + U*encoder_output) --> (32,20,256)
    # hidden_layer --> (32,1,256)
    # encoder_output --> (32,20,256)
    out = self.tanh(self.Wa(L) + self.Ua(encoder_outputs))
    # print("DecoderModel:weighted out :",out.shape)

    # (32x20x256) x (32x256x1) ---> (32x20x1) --> (32x20)
    wts = torch.bmm(out, nn.Parameter(torch.FloatTensor(batch_size, self.hidden_size)).unsqueeze(2)).squeeze(-1)
    # print("DecoderModel:weights :",wts.shape)

    # Weighted sum of encoder outputs (32x1x20) x (32x20x256) --> (32x1x256) --> (32x256)
    context = torch.bmm(wts.unsqueeze(1), encoder_outputs).squeeze(1)
    # print("DecoderModel:context :",context.shape)

    # Linearize context and attn_comb(Embedding, Previous hidden layer)
    # step 1 : (32x256) + (32x256) --> (32x512)
    # step 2 : (32x512) --> (32x256)
    predictions = self.softmax(self.fc(torch.cat((L.squeeze(1), context),1)))
    # print("DecoderModel:Predictions : ",predictions.shape)

    return predictions, hidden_state

input_size_decoder = len(characters)
decoder_embedding_size = 300
hidden_size = 128
num_layers = 2
decoder_dropout = float(0.5)
output_size = len(characters)
max_length = 20

decoder_gru = AttentionDecoderGRU(input_size_decoder, decoder_embedding_size,
                           hidden_size, max_length, num_layers, decoder_dropout, output_size)
print(decoder_gru)

In [ ]:
class Seq2SeqGRU(nn.Module):
  def __init__(self, Encoder_GRU, Decoder_GRU):
    super(Seq2SeqGRU, self).__init__()
    self.Encoder_GRU = Encoder_GRU
    self.Decoder_GRU = Decoder_GRU

  def forward(self, source, target, num_layers,hidden_size,max_length = 20, tfr=0.5):
    # Shape - Source : (20, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (20, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(characters)
    
    # Shape --> outputs (20, 32, 66) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size)
    # print("Model:Output : ",outputs.shape)

    temp_encoder_outputs = torch.zeros(max_length, batch_size, hidden_size)
    # print("Model:Temp Encoder_outputs",temp_encoder_outputs.shape)

    encoder_outputs = torch.zeros(batch_size,max_length, hidden_size)
    # print("Model:Encoder_outputs",encoder_outputs.shape)

    hidden_state_encoder = torch.zeros(num_layers, batch_size, hidden_size) # 1 X 32 X 256
    # print("Model:InitStateLayer : ",hidden_state_encoder.shape,cell_state_encoder.shape)

    # Shape --> (hs, cs) (2, 32, 256) ,(2, 32, 256) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    for ei in range(target_len):
        encoder_output, hidden_state_encoder = self.Encoder_GRU(source[ei], hidden_state_encoder)
        temp_encoder_outputs[ei] = encoder_output[0]
        # print(ei," Model:Encoder Outputs and states :")
        # print(encoder_output.shape)
        # print(hidden_state_encoder.shape)
        # print(cell_state_encoder.shape)
    
    for i in range(batch_size):
      for j in range(max_length):
        encoder_outputs[i][j] = temp_encoder_outputs[j][i]
    
    # hidden_state_encoder, cell_state_encoder = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>
    # print("Model:Target : ", x.shape)

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state_decoder = self.Decoder_GRU(x, hidden_state_encoder, encoder_outputs,batch_size)
      # print(ei," Model:Decoder Outputs and states :")
      # print(output.shape)
      # print(hidden_state_decoder.shape)
      # print(cell_state_decoder.shape)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs

model_gru = Seq2SeqGRU(encoder_gru,decoder_gru)
print(model_gru)

In [ ]:
# epoch_loss = 0.0
# num_epochs = 20
# best_loss = 999999
# best_epoch = -1
# for batch in range(num_of_batches):
#   input = torch.tensor(x_train[32*batch:32*(batch+1)][:]).T # of size 20 X 32 (32 -> batch size, 20 -> 1 word max letters)
#   target = torch.tensor(y_train[32*batch:32*(batch+1)][:]).T # of size 20 X 32
#   output = model_lstm(input, target,1,128)
#   # print(output.shape[0]-1)
#   output = output[:output.shape[0]-1].reshape(-1, output.shape[2])
#   target = target[1:].reshape(-1)
#   # print(output.shape, target.shape)

#   # Clear the accumulating gradients
#   optimizer.zero_grad()
#   # Calculate the loss value for every epoch
#   loss = criterion(output, target)

#   # Calculate the gradients for weights & biases using back-propagation
#   loss.backward()
#   # Clip the gradient value is it exceeds > 1
#   torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1)
#   # Update the weights values using the gradients we calculated using bp 
#   optimizer.step()
#   # step += 1
#   # epoch_loss += loss.item()
#   # writer.add_scalar("Training loss", loss, global_step=step)

# if epoch_loss < best_loss:
#   best_loss = epoch_loss
#   # best_epoch = epoch
#   # save_checkpoint(dict({'model':model, 'best_loss':best_loss, 'epoch':epoch, 'optimizer':optimizer, 'epoch_loss':epoch_loss}))
#   # if ((epoch - best_epoch) >= 10):
#     # print("no improvement in 10 epochs, break")
# print("Epoch_Loss - {}".format(loss.item()))
# print()

In [ ]:
# result = translate_word(model_lstm, x_train[0], y_train[0],1)
# print(result)